In [1]:
import sys
sys.path.insert(1, '../Codes_1Dtree')
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

Load libraries

In [2]:
import torch
from torch_geometric.loader import DataLoader
from data.graph_dataset import OneDDatasetBuilder, OneDDatasetLoader, normalize
# from networks.gcn import GraphUNet, RecurrentFormulationNet
from networks.gcnv5 import RecurrentFormulationNet
import matplotlib.pyplot as plt
from torch_geometric.loader import NeighborLoader
from networks.losses import LpLoss

In [3]:
class objectview(object):
    def __init__(self, d) -> None:
        self.__dict__ = d
    def setattr(self, attr_name, attr_value):
        self.__dict__[attr_name] = attr_value

args = objectview({
    'n_field': 1,
    'n_meshfield': (3, 13),
    'hidden_size': 256,
    'n_hidden': 10,
    'n_time': 2,
    'aggr': 'sum',
    'act': 'leakyrelu',
    'dropout': 0.1,
    'use_hidden': True,
    'device': torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'),
    'lr': 1e-4,
    # 'lr_decay': 0.5,
    # 'lr_decay_period': 50,
    'weight_decay': 1e-3,
    'n_epoch': 10000,
    'alpha': 1.0,
    'batchsize': 10000,
    'timestep': 201,
    'timeslice_hops': 0,
    'timeslice_steps': 1,
    'n_data_per_batch': 1,
    'forward_sequence':False,
    'criterion': LpLoss(),
    'plot': False
})

Load dataset

In [4]:
# dataset = OneDDatasetBuilder(
#     raw_dir='/data1/tam/datasets',
#     root_dir='/data1/tam/downloaded_datasets_edge_node_separated',
#     sub_dir='processed',
#     subjects='all',
#     time_names=[str(i).zfill(3) for i in range(201)],
#     data_type = torch.float32,
#     readme='edge_index(2xn_edge), node_attr(n_nodex10), pressure+flowrate(n_nodex201)'
# )
dataset = OneDDatasetLoader(
    root_dir='/data1/tam/downloaded_datasets_edge_node_separated',
    sub_dir='processed',
    subjects='all',
    time_names=[str(i).zfill(3) for i in range(201)],
    data_type = torch.float32
)

In [5]:
# dataset = normalize(
#     dataset=dataset,
#     sub_dir='normalized',
#     scaler_dict={
#         'node_attr': ('minmax_scaler', 0, None),
#         'edge_attr': ('minmax_scaler', 0, None),
#         'pressure': ('minmax_scaler', None, None),
#         # 'flowrate': ('robust_scaler', None, None),
#         # 'pressure_dot': ('minmax_scaler', None, None),
#         # 'flowrate_dot': ('robust_scaler', None, None),
#         # 'time': ('minmax_scaler', None, None)
#     }
# )
dataset = OneDDatasetLoader(
    root_dir='/data1/tam/downloaded_datasets_edge_node_separated',
    sub_dir='normalized',
    subjects='all',
    time_names=[str(i).zfill(3) for i in range(201)],
    data_type = torch.float32
)

In [6]:
# dataset = batchgraph_generation_wise(
#     dataset,
#     sub_dir='batched',
#     batch_gens=[[14,15]],
#     timestep=args.timestep,
#     timeslice_hops=args.timeslice_hops,
#     timeslice_steps=args.timeslice_steps
# )
# dataset = batchgraph(
#     dataset,
#     sub_dir='batched_1',
#     batchsize=None,
#     timestep=args.timestep,
#     timeslice_hops=args.timeslice_hops,
#     timeslice_steps=args.timeslice_steps
# )
# dataset = OneDDatasetLoader(
#     root_dir='/data1/tam/downloaded_datasets_Static_v1',
#     sub_dir='batched_1',
#     subjects='all',
#     time_names=[str(i).zfill(3) for i in range(201)],
#     data_type = torch.float32
# )

In [7]:
# fold_size = 2
# fold = [list(range(i*fold_size, (i+1)*fold_size)) for i in range(8)]
# n_train_fold = 5

# for i in range(n_train_fold):
#     test_subset = fold[i]
#     train_subset = []
#     for j in range(n_train_fold):
#         if j != i:
#             train_subset += fold[j]

# (train_loader, test_loader) = dataset_to_loader(
#     dataset=dataset,
#     data_subset_dict={
#         'train': list(range(5, 6)),
#         'test': list(range(0, 1))
#     },
#     n_data_per_batch=args.n_data_per_batch
# )

# train_set, test_set = dataset_to_loader(
#     dataset=dataset,
#     data_subset_dict={
#         'train': list(range(6, 36)),
#         'test': list(range(0, 5))
#     },
#     n_data_per_batch=args.n_data_per_batch
# )

Model initializing and training

In [8]:
model = RecurrentFormulationNet(
    n_field=args.n_field,
    n_meshfield=args.n_meshfield,
    hidden_size=args.hidden_size,
    n_hidden=args.n_hidden,
    act=args.act,
    dropout=args.dropout,
    use_hidden=args.use_hidden
)
setattr(model, 'name', 'model_GCN')
model = model.to(args.device)
# model.load_state_dict(torch.load(f'models/{model.name}_node2_epoch200.pth', map_location=args.device) )
# optimizer1 = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
# optimizer2 = torch.optim.LBFGS(model.parameters(), lr=args.lr)
# setattr(args, 'optimizer', optimizer2)
# setattr(args, 'optimizer2', optimizer2)

In [9]:
def train(model, data, args):
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
    ##
    F_true = torch.cat([data.pressure.unsqueeze(2)], dim=2) \
                .float().to(args.device)
    ##
    if not args.forward_sequence:
        F_0 = F_true[:,0,:]
    else:
        F_0 = F_true[:,:-1,:]
    edge_index = data.edge_index.to(args.device)
    # edge_index = torch.cat([data.edge_index, torch.flip(data.edge_index, dims=[0])], dim=1).to(args.device)
    meshfield = (data.node_attr.float().to(args.device), data.edge_attr.float().to(args.device))
    # meshfield = (data.node_attr.float().to(args.device), torch.cat([data.edge_attr, data.edge_attr], dim=0).float().to(args.device))
    F_true = F_true[:,1:args.n_time,:]
    ##
    F_pred = model.forward(
        F_0=F_0,
        edge_index=edge_index,
        meshfield=meshfield,
        n_time=args.n_time-1
    )
    ##
    optimizer.zero_grad()
    model.train()
    loss = args.criterion(F_pred, F_true)
    loss.backward()
    optimizer.step()
    
    return loss.item()

def eval(model, data, args):
    ##
    F_true = torch.cat([data.pressure.unsqueeze(2)], dim=2) \
                .float().to(args.device)
    ##
    if not args.forward_sequence:
        F_0 = F_true[:,0,:]
    else:
        F_0 = F_true[:,:-1,:]
    edge_index = data.edge_index.to(args.device)
    # edge_index = torch.cat([data.edge_index, torch.flip(data.edge_index, dims=[0])], dim=1).to(args.device)
    meshfield = (data.node_attr.float().to(args.device), data.edge_attr.float().to(args.device))
    # meshfield = (data.node_attr.float().to(args.device), torch.cat([data.edge_attr, data.edge_attr], dim=0).float().to(args.device))
    F_true = F_true[:,1:args.n_time,:]
    ##
    with torch.no_grad():
        F_pred = model.forward(
            F_0=F_0,
            edge_index=edge_index,
            meshfield=meshfield,
            n_time=args.n_time-1
        )
    ##
    loss = args.criterion(F_pred, F_true)
        
    return loss.item()

In [10]:
train_subset = list(range(0, 10))
test_subset = list(range(20,30))

In [11]:
# Training
train_loss = 0
eval_loss = 0
for epoch in range(args.n_epoch):
    CUDA_LAUNCH_BLOCKING=1
    torch.cuda.empty_cache()
    ##
    if epoch % 100 == 0:
        if args.n_time < 100:
            args.n_time *= 2
    # train_loss = 0
    for i_data in train_subset:
        data = dataset[i_data]
        if args.batchsize is not None:
            train_loader = NeighborLoader(data, num_neighbors=[1], batch_size=args.batchsize)
            for i in range(train_loader.__len__()):
                data = next(iter(train_loader))
                train_loss += train(model=model, data=data, args=args) / train_loader.__len__()
        else:
            train_loss += train(model=model, data=data, args=args)

    # eval_loss = 0
    for i_data in test_subset:
        data = dataset[i_data]
        if args.batchsize is not None:
            test_loader = NeighborLoader(data, num_neighbors=[1], batch_size=args.batchsize)
            for i in range(test_loader.__len__()):
                data = next(iter(test_loader))
                eval_loss += eval(model=model, data=data, args=args) / test_loader.__len__()
        else:
            eval_loss += eval(model=model, data=data, args=args)
    
    print(f'Epoch {epoch}: train loss = {train_loss}; eval loss = {eval_loss}')
    train_loss = 0
    eval_loss = 0
    # else:
        # print(f'Epoch {epoch}.')
        
    if (epoch+1) % 20 == 0:
        torch.save(model.state_dict(), f'models/{model.name}_node2_epoch{epoch+1}.pth')

/home/tam/.conda/envs/tam_geometric/lib/python3.11/site-packages/torch_geometric/utils/sparse.py:268: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  adj = torch.sparse_csr_tensor(


Epoch 0: train loss = 2.1503426109751076; eval loss = 0.33225364956472586
Epoch 1: train loss = 0.4484237944707274; eval loss = 0.3209211368424196
Epoch 2: train loss = 0.42690608945364755; eval loss = 0.5131567030524212
Epoch 3: train loss = 0.39460279063011194; eval loss = 0.2183445394039155
Epoch 4: train loss = 0.38716680835932493; eval loss = 0.21350127811144498
Epoch 5: train loss = 0.3806394819791119; eval loss = 0.20399870115908822
Epoch 6: train loss = 0.3816278555120031; eval loss = 0.32272220325345763
Epoch 7: train loss = 0.3716800686282416; eval loss = 0.49275097211024593
Epoch 8: train loss = 0.3698796476237477; eval loss = 0.46380399912595743
Epoch 9: train loss = 0.36606555556257564; eval loss = 0.4854874399801096
Epoch 10: train loss = 0.3683885036346813; eval loss = 0.5005667395889759
Epoch 11: train loss = 0.3683725016502043; eval loss = 0.3015189155315357
Epoch 12: train loss = 0.3639293443411588; eval loss = 0.42723002936691035
Epoch 13: train loss = 0.354014085140

KeyboardInterrupt: 